# Introduction
In [this notebook](https://www.kaggle.com/dliend/fastai-tabular-learner-moa-challenge), we used a basic fastai `TabularLearner` to generate predictions for this challenge. We did not attempt substantial hyperparameter tuning (aside from tweaking the learning rate).

In [subsequent approaches](https://www.kaggle.com/dliend/testing-moa-based-on-grid-search-results), we used grid search and a lot of manual hyperparameter tuning and found that wide, two-layer networks with a fair amount of regularization and small batch sizes performed better than alternatives.

In this notebook, we attempt to make greater use of the training data available to us by using cross validation. We will be following examples set up on other notebooks, which will be referenced here.

In particular, we start by following the cross validation example implemented [here](https://www.kaggle.com/robertlangdonvinci/lish-moa-kfold-fastai-tabnet-ensemble).

In [1]:
import numpy as np
import pandas as pd

# Define MultilabelStratifiedKFold Class
SOURCE: https://www.kaggle.com/robertlangdonvinci/lish-moa-kfold-fastai-tabnet-ensemble/data?select=ml_stratifiers.py  (should track down original source)

In [2]:
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, \
    BaseShuffleSplit, _validate_shuffle_split
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target
from sklearn.utils import check_random_state



def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds

class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits, shuffle, random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)

In [3]:
from fastai.tabular.all import *
path = Path('../input/lish-moa')
train_features = pd.read_csv(path/'train_features.csv')
test_features = pd.read_csv(path/'test_features.csv')
train_targets = pd.read_csv(path/'train_targets_scored.csv')
train_drugs = pd.read_csv(path/'train_drug.csv')
sample_submission = pd.read_csv(path/'sample_submission.csv')

From the documentation:

>Features for the training set. Features `g-` signify gene expression data, and `c-` signify cell viability data. `cp_type` indicates samples treated with a compound (`cp_vehicle`) or with a control perturbation (`ctrl_vehicle`); control perturbations have no MoAs; `cp_time` and `cp_dose` indicate treatment duration (24, 48, 72 hours) and dose (high or low).

# Data Preprocessing

In [4]:
categorical = ['cp_type', 'cp_time', 'cp_dose']
continuous = [i for i in train_features.columns if i not in ['cp_type', 'cp_time', 'cp_dose', 'sig_id']]
dep_var = [i for i in train_targets.columns if i != 'sig_id']
train_features[dep_var] = train_targets[dep_var]
train_features.shape

(23814, 1082)

## Set up Folds

In [5]:
df = train_features.sample(frac=1.,random_state=41)
df['kfold'] = -1
kf = MultilabelStratifiedKFold(n_splits=10)
y = df[dep_var].values
for fold, (t_,v_) in enumerate(kf.split(X=df,y=y)):
    df.loc[v_,'kfold'] = fold

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=False, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [6]:
procs = [FillMissing, Categorify, Normalize]
def get_data(fold):
    val_idx = df[df.kfold==fold].index
    dls = TabularDataLoaders.from_df(df, path=path, y_names=dep_var,
                                        cat_names = categorical,
                                        cont_names = continuous,
                                        procs = procs,
                                        valid_idx=val_idx,
                                        bs=64)
    return dls

# Fit Model


In [7]:
torch.cuda.empty_cache() 
test_sc = []
best_valid = []

for i in range(10):
    
    dls = get_data(i) # Data
    
    learn = tabular_learner(dls, layers=[15000, 1600], loss_func = BCEWithLogitsLossFlat(),
                            model_dir='/kaggle/working/', wd = 4) # Model
    
    name = 'best_model_' + str(i) 
    cb = SaveModelCallback(monitor='valid_loss',fname=name ,mode='min') # Callbacks
    
    lr = .003
    learn.fit_one_cycle(19, lr,cbs=cb) # Training
    
    learn.load(name) # Load best model
    losses = np.array(learn.recorder.values)
    best = np.argmin(losses[:,1])
    best_valid.append(losses[best,1])
    
    test_dl = learn.dls.test_dl(test_features)
    sub = learn.get_preds(dl=test_dl) # prediction
    test_sc.append(sub[0].numpy())
    
    learn.export('/kaggle/working/'+name+'.pkl') # export model
    
test_sc = np.array(test_sc)
best_valid

epoch,train_loss,valid_loss,time
0,0.018027,0.017530,00:07
1,0.017834,0.018819,00:06
2,0.021846,0.019682,00:06
3,0.020238,0.019569,00:06
4,0.019587,0.018910,00:06
5,0.018719,0.018206,00:06
6,0.018455,0.018378,00:06
7,0.018014,0.018905,00:07
8,0.017875,0.017889,00:06
9,0.017609,0.017110,00:06


Better model found at epoch 0 with valid_loss value: 0.017530489712953568.
Better model found at epoch 9 with valid_loss value: 0.01710977591574192.
Better model found at epoch 10 with valid_loss value: 0.016927633434534073.
Better model found at epoch 12 with valid_loss value: 0.016229091212153435.
Better model found at epoch 13 with valid_loss value: 0.016059191897511482.
Better model found at epoch 14 with valid_loss value: 0.015406939201056957.
Better model found at epoch 15 with valid_loss value: 0.014923859387636185.
Better model found at epoch 16 with valid_loss value: 0.014655360952019691.
Better model found at epoch 17 with valid_loss value: 0.014516839757561684.
Better model found at epoch 18 with valid_loss value: 0.014504811726510525.


epoch,train_loss,valid_loss,time
0,0.018101,0.017058,00:06
1,0.017868,0.017829,00:07
2,0.021890,0.019210,00:07
3,0.020071,0.018699,00:06
4,0.019110,0.019028,00:06
5,0.019157,0.018855,00:06
6,0.018839,0.018290,00:07
7,0.018471,0.017634,00:06
8,0.018184,0.017335,00:06
9,0.017571,0.016904,00:06


Better model found at epoch 0 with valid_loss value: 0.01705787144601345.
Better model found at epoch 9 with valid_loss value: 0.01690361462533474.
Better model found at epoch 11 with valid_loss value: 0.016352146863937378.
Better model found at epoch 12 with valid_loss value: 0.0161129217594862.
Better model found at epoch 14 with valid_loss value: 0.015358890406787395.
Better model found at epoch 15 with valid_loss value: 0.014912722632288933.
Better model found at epoch 16 with valid_loss value: 0.01454931776970625.
Better model found at epoch 17 with valid_loss value: 0.014365144073963165.
Better model found at epoch 18 with valid_loss value: 0.014327154494822025.


epoch,train_loss,valid_loss,time
0,0.018164,0.017225,00:06
1,0.017568,0.017426,00:06
2,0.021621,0.020797,00:07
3,0.019470,0.018736,00:06
4,0.019479,0.019323,00:06
5,0.019020,0.018810,00:07
6,0.018641,0.018589,00:07
7,0.018047,0.018558,00:07
8,0.017904,0.017588,00:06
9,0.017399,0.017018,00:06


...

epoch,train_loss,valid_loss,time
0,0.017877,0.016947,00:07
1,0.017566,0.017943,00:07
2,0.020689,0.022197,00:06
3,0.019878,0.020006,00:07
4,0.019270,0.018663,00:07
5,0.018785,0.018886,00:06
6,0.018279,0.018097,00:07
7,0.018182,0.018253,00:07
8,0.017573,0.017145,00:07
9,0.017723,0.018252,00:07


Better model found at epoch 0 with valid_loss value: 0.01694653555750847.
Better model found at epoch 11 with valid_loss value: 0.01674657315015793.
Better model found at epoch 12 with valid_loss value: 0.016215967014431953.
Better model found at epoch 13 with valid_loss value: 0.016098739579319954.
Better model found at epoch 14 with valid_loss value: 0.01577601209282875.
Better model found at epoch 15 with valid_loss value: 0.015083731152117252.
Better model found at epoch 16 with valid_loss value: 0.014869291335344315.
Better model found at epoch 17 with valid_loss value: 0.014669515192508698.


epoch,train_loss,valid_loss,time
0,0.018197,0.017132,00:07
1,0.017927,0.018536,00:07
2,0.021623,0.020799,00:07
3,0.020122,0.018424,00:07
4,0.018976,0.019064,00:07
5,0.018965,0.018366,00:06
6,0.018610,0.018298,00:07
7,0.018401,0.018165,00:07
8,0.018019,0.018731,00:06
9,0.017369,0.017655,00:07


Better model found at epoch 0 with valid_loss value: 0.017132405191659927.
Better model found at epoch 10 with valid_loss value: 0.01662501133978367.
Better model found at epoch 13 with valid_loss value: 0.01572243496775627.
Better model found at epoch 14 with valid_loss value: 0.01547402236610651.
Better model found at epoch 15 with valid_loss value: 0.01506544928997755.
Better model found at epoch 16 with valid_loss value: 0.014729545451700687.
Better model found at epoch 17 with valid_loss value: 0.01454158779233694.


[0.014504811726510525,
 0.014327154494822025,
 0.014626939781010151,
 0.014706913381814957,
 0.014544399455189705,
 0.014812259934842587,
 0.014692756347358227,
 0.014432634226977825,
 0.014669515192508698,
 0.01454158779233694]

In [8]:
np.argmin(best_valid)

1

# Get Predictions for Submission
We follow the guide to setting up a test set here: https://forums.fast.ai/t/a-brief-guide-to-test-sets-in-v2-you-can-do-labelled-now-too/57054

In [9]:
avg_prds = test_sc[np.argmin(best_valid)]
submission = sample_submission.copy()
submission[dep_var] = avg_prds
submission.loc[test_features['cp_type']=='ctl_vehicle', dep_var] = 0
submission['atp-sensitive_potassium_channel_antagonist'] = 0 # only appears once
submission['erbb2_inhibitor'] = 0 # only appears once

In [10]:
submission.to_csv('submission.csv', index=False)